In [2]:
from IPython.display import HTML

HTML('''
<style>
    .jp-InputPrompt, .jp-Cell-inputWrapper {
        display: block !important;
    }
</style>
''')


In [ ]:
from IPython.display import HTML

HTML('''
<style>
    .jp-InputPrompt, .jp-Cell-inputWrapper {
        display: none !important;
    }
</style>
''')


In [1]:
import os
import re
from IPython.display import display, HTML

# Define dataset structure
scenario_map = {
    'UCI': ['3111', '222', '231', '321', '33', '42']
}
persons = ["P0_Results", "P1_Results", "P2_Results"]
acc_types = ["All_Acc", "New_Acc", "Old_Acc"]

# Function to find the latest task number from boxplot filenames
def find_latest_task(folder_path):
    boxplot_files = [f for f in os.listdir(folder_path) if re.match(r".*_Box_plot_Acc_Task_\d+\.png$", f)]
    if not boxplot_files:
        return None
    latest_file = max(boxplot_files, key=lambda f: int(re.search(r"Task_(\d+)", f).group(1)))
    return latest_file

# Loop through datasets, scenarios, and persons
for dataset in list(scenario_map.keys()):
    scenarios = scenario_map.get(dataset, [])
    for scenario in scenarios:
        for person in persons:
            html_content = f"<h3>{dataset} - {scenario} - {person}</h3>"
            html_content += '<table style="border-collapse: collapse;"><tr>'

            # Collect images for this scenario & person
            row1 = []  # First row (Boxplot + All_Acc Barchart)
            row2 = []  # Second row (New_Acc + Old_Acc Barcharts)

            for acc in acc_types:
                folder_path = os.path.abspath(f"{dataset}/{scenario}/{person}/Figures/{acc}")

                if not os.path.exists(folder_path):
                    print(f"❌ Folder Not Found: {folder_path}")  # Debug
                    continue

                # Find the boxplot for All_Acc (only need one)
                if acc == "All_Acc":
                    latest_boxplot_file = find_latest_task(folder_path)
                    if latest_boxplot_file:
                        boxplot_path = os.path.join(folder_path, latest_boxplot_file)
                        rel_boxplot_path = os.path.relpath(boxplot_path, os.getcwd())  # Relative path
                        row1.append(f'<td><img src="{rel_boxplot_path}" width="400"></td>')

                # Find the bar chart for each acc type
                barchart_file = next((f for f in os.listdir(folder_path) if f.endswith("Acc.png") and "Box_plot" not in f), None)
                if barchart_file:
                    barchart_path = os.path.join(folder_path, barchart_file)
                    rel_barchart_path = os.path.relpath(barchart_path, os.getcwd())  # Relative path

                    if acc == "All_Acc":
                        row1.append(f'<td><img src="{rel_barchart_path}" width="400"></td>')
                    elif acc == "New_Acc":
                        row2.append(f'<td><img src="{rel_barchart_path}" width="400"></td>')
                    elif acc == "Old_Acc":
                        row2.append(f'<td><img src="{rel_barchart_path}" width="400"></td>')

            # Ensure exactly 2 images per row
            while len(row1) < 2:
                row1.append("<td></td>")  # Empty cell if missing images
            while len(row2) < 2:
                row2.append("<td></td>")  # Empty cell if missing images

            # Combine rows into HTML
            html_content += "<tr>" + "".join(row1) + "</tr>"
            html_content += "<tr>" + "".join(row2) + "</tr>"
            html_content += "</table><br>"

            # Display the formatted layout
            display(HTML(html_content))
